## Why is my computer slow?

The general strategy for addressing slowness is to identify the **BottleNeck** for addressing slowness in our device, our script, or our system to run slowly. 

The bottleneck could be the CPU time as we just mentioned. But it could also be time spent reading data from disk waiting for data transmitted over the network, moving data from disk to RAM, or some other resource that's limiting the overall performance.

This only helps us if the issue is that there are too many processes trying to use the same resource. If we've closed everything that wasn't needed and the computer is still slow, we need to look into other possible explanations.

### Hardware Problems

What if the hardware we're using just isn't good enough for the applications we're trying to run on it? In cases like these, will have to upgrade the underlying hardware.

But to make a difference in the resulting performance, we need to make sure that we're actually improving the bottleneck and not just wasting our money on new hardware that will go unused. So how can we tell which piece of hardware needs to be changed? We need to monitor the usage of our resources to know which of them as being exhausted. 


### For Linux

To find out, we use the tools available in our operating system, monitor the usage of each resource, and then work out which one is blocking our programs for running faster. We've already talked about using **top on Linux systems**. This tool lets us see which currently running processes are using the most CPU time. If we start by memory, which ones are using the most memory. It also shows a bunch of other load information related to the current state of the computer, like how many processes are running and how the CPU time or memory is being used.


We also called out in earlier videos a couple of other programs like iotop and iftop. They can help us see which processes are currently using the most disk IO usage or the most network bandwidth. 


### For MacOs and Windows Users

On MacOS, the OS ships with a tool called Activity Monitor which lets us see what's using the most CPU, memory, energy, disk, or network. On Windows, there's a couple of OS tools called Resource Monitor and Performance Monitor which also let us analyze what's going on with the different resources on the computer including CPU, memory, disk and network.


## How Computer Use Resources

if you can create a cache, **a cache** stores data in a form that's faster to access than its original form. There's a ton of examples of caches in IT. A web proxy is a form of cash. It stores websites, images, or videos that are accessed often by users behind the proxy. So they don't need to be downloaded from the Internet every time.

 If there's still not enough RAM after that, the operating system will put the parts of the memory that aren't currently in use onto the hard drive in a space called **swap**. Reading and writing from disk is much slower than reading and writing from RAM. 
 
The swapping implementation varies across the different operating systems, but the concept is always the same. The information that's not needed right now is removed from RAM and put onto the disk, while the information that's needed now is put into RAM. 

So what do you do if you find that your machine is slow because it's spending a lot of time swapping? There are basically three possible reasons for this.

1- First, if there are too many open applications and some can be closed, close the ones that aren't needed. 

2- if the available memory is just too small for the amount that computer is using, add more RAM to the computer. 

3- The third reason is that one of the running programs may have a memory leak, causing it to take all the available memory. **A memory leak** means that memory which is no longer needed is not getting released.



## Possible Causes of Slowness

. If the file is a log file, you can use a program like logrotate to do this for you

## Slow Web Server

### Apache Benchmark tool

use a tool called **ab which stands for Apache Benchmark tool** to figure out how slow it is. We'll run **ab -n 500 websiteAddres** to get the average timing of 500 requests, and then pass our site.example.com for the measurement. This tool is super useful for checking if a website is behaving as expected or not. It will make a bunch of requests and summarize the results once it's done. Here, were asking for it to do 500 requests to our website.

Connecting to webserver
**ssh webserver**

#### To reduce the load

 One thing we can try is to change the processes priorities so that the web server takes precedence. The process priorities in Linux are so that the lower the number, the higher the priority. **Typical numbers go from 0 to 19**. By default, processes start with a priority of zero.
 
**nice and renice command**

But we can change priorities using the nice and renice commands. 

We use nice for starting a process with a different priority and 

renice for changing the priority of a process that's already running


We want to run renice for all the ffmpeg processes that are running right now. We could do this one by one. But it would be manual, error-prone, and super boring. Instead, we can use a quick line of shell script to do this for us.

**pidof**

For that, we'll use the pidof command that receives the process name and returns all the process IDs that have that name. 

We'll iterate over the output of the pidof command with a for loop and then call renice for each of the process IDs. 

Renice takes the new priority as the first argument, and the process ID to change as the second one. In our case, we'll want the lowest possible priority which is 19.

So we'll call 

```
for pid in $(pidof ffmpeg);

do renice 19 $pid; done

```
All right. We see that the priorities for those processes were updated.

## still slower website

Apparently, the OS is still giving these ffmpeg processes way too much processor time. Our website is still slow. What else can we do? These transcoding processes are CPU intensive, and running them in parallel is overloading the computer. 

So one thing we could do is, modify whatever's triggering them to run them one after the other instead of all at the same time.

To do that, we'll need to find out how these processes got started. First, we'll look at the output of the **ps command to get some more information about the processes.**

We'll call **ps ax which shows us all the running processes on the computer**, and we'll connect the output of the command to less, to be able to scroll through it.

Now we'll look for the ffmpeg process using slash which is the search key when using less.

```
ps ax | less


/ffmpeg

```
 We can try using the locate command to see if we can find them. We'll first exit the less interface with queue and then call locate static/001.webm. We see that the static directory is located in the server deploy videos directory.
 
 ```
     locate static/001.webm
 
 ```
 
 There's a bunch of files here. We could check them all one-by-one to see if one of them contained a call to ffmpeg. But that sounds like a lot of manual work. Instead, let's use grep to check if any of these files contains a call to ffmpeg.
 
 ```
 grep ffmpeg *
 
 ```
 
 #### Daemonize 
 
 We see that this script is starting the ffmpeg processes in parallel using a tool called Daemonize that runs each program separately as if it were a daemon. This might be okay if we only need to convert a couple of videos but launching one separate process for each of the videos in the static directory is overloading our server. So we want to change this to run only one video conversion process at a time. We'll do that by simply deleting the daemonized part and keeping the part that calls ffmpeg, then save and exit.
 
 ```
 remove (daemon -c PWD)
 ```
 
 But this won't change the processes that are already running. **We want to stop these processes but not cancel them completely**, as doing so would mean that the videos being converted right now will be incomplete. So we'll use the **killall command with the -STOP flag which sends a stop signal but doesn't kill the processes completely**.
 
 We now want to run these processes one at a time. How can we do that? We could send the **CONT signal** to one of them, wait till it's done, and then send it to the next one. But that's a lot of manual work. Can be automate it? Yes. But it's a little tricky. So pay close attention. We can iterate through the list of processes using the same for loop with the **pit of command** that we used earlier.
 
 This will succeed as long as the process exists, and fails once the process goes away. Inside this while loop, we'll simply add a call to sleep one, to wait one second until the next check.

```
    killall -STOP ffmpeg
    
    for pid in $(pidof ffmpeg); do while kill -CONT $pid; do sleep 1; done; done
```

Now our server is running one ffmpeg process at a time.